In [1]:
from CSVUtils import *
import pandas as pd
import numpy as np
from os import path

In [2]:
DIR = "./input/currency"
name_prefix = ["USD_TWD", "BRL_USD"]
name_suffix = ['1995-2009','2010-2020']
df_list = []
for suffix in name_suffix:
    filename = "USD_TWD Historical Data_"+suffix+".csv"
    df = csv2df(DIR, filename, source="investing")
    df['Price'] = 1/df['Price']
    df['Open'] = 1/df['Open']
    df['High'] = 1/df['High']
    df['Low'] = 1/df['Low']
    df['Change'] = df['Price'].pct_change()
    df = df.sort_values('Date').reset_index(drop=True)
    df_list.append(df)

In [3]:
df[df['Date'] == pd.to_datetime("2015-08-25")]

,Date,Price,Open,High,Low,Change
1472,2015-08-25,0.030813,0.030298,0.030207,0.030916,0.000555


In [4]:
twd_df = pd.concat(df_list)
twd_df = twd_df.dropna()
twd_df.reset_index(drop=True,inplace=True)
twd_df

,Date,Price,Open,High,Low,Change
0,1995-01-02,0.038124,0.038124,0.038124,0.038124,0.000000
1,1995-01-03,0.038124,0.038124,0.038124,0.038124,0.000000
2,1995-01-04,0.038124,0.038124,0.038124,0.038124,0.004346
3,1995-01-05,0.037959,0.038146,0.037959,0.038146,-0.001291
4,1995-01-06,0.038008,0.038081,0.037913,0.038081,0.002090
...,...,...,...,...,...,...
6556,2020-02-26,0.033023,0.032860,0.032837,0.033053,-0.000198
6557,2020-02-27,0.033029,0.033021,0.032908,0.033151,-0.004591
6558,2020-02-28,0.033182,0.033056,0.032929,0.033225,-0.009025
6559,2020-03-02,0.033484,0.033182,0.033120,0.033494,0.000670


In [5]:
DIR = "./input/currency"
name_suffix = ['1995-2009','2010-2020']
df_list = []
prefix = "BRL_USD Historical Data_"
for suffix in name_suffix:
    filename = prefix+suffix+".csv"
    df = csv2df(DIR, filename, source="investing")
    df = df.sort_values('Date').reset_index(drop=True)
    df_list.append(df)

In [6]:
brl_df = pd.concat(df_list)
brl_df = brl_df.dropna()
brl_df.reset_index(drop=True,inplace=True)
brl_df

,Date,Price,Open,High,Low,Change
0,1995-01-02,9.3123,9.3020,9.3123,9.3020,-0.0072
1,1995-01-03,9.3106,9.3003,9.3106,9.3003,-0.0002
2,1995-01-04,9.3518,9.3415,9.3518,9.3415,0.0044
3,1995-01-05,9.2882,9.2780,9.2882,9.2780,-0.0068
4,1995-01-06,9.2929,9.2878,9.2929,9.2878,0.0005
...,...,...,...,...,...,...
6494,2020-02-26,0.2247,0.2278,0.2279,0.2244,-0.0140
6495,2020-02-27,0.2229,0.2247,0.2248,0.2221,-0.0080
6496,2020-02-28,0.2235,0.2220,0.2240,0.2215,0.0027
6497,2020-03-02,0.2235,0.2225,0.2238,0.2219,0.0000


In [7]:
twd_df.to_csv("./input/currency/TWD_1995-2020.csv", index=False)
brl_df.to_csv("./input/currency/BRL_1995-2020.csv", index=False)

In [8]:
DIR = "./from github\Stock-Trading-Environment\data"
twii_df = csv2df(DIR, "^TWII.csv", source="yahoo")
bvsp_df = csv2df(DIR, "^BVSP.csv", source="yahoo")

In [9]:
twd_df = pd.read_csv("./input/currency/TWD_1995-2020.csv")
brl_df = pd.read_csv("./input/currency/BRL_1995-2020.csv")

In [10]:
twd_df['Date'] = pd.to_datetime(twd_df['Date'])
brl_df['Date'] = pd.to_datetime(brl_df['Date'])

In [11]:
def normalize_to_usd(market_df, currency_df):
    currency_df = currency_df[['Date', 'Price']]
    common_date = np.intersect1d(currency_df['Date'] , market_df['Date'])
    currency_df = currency_df[currency_df['Date'].isin(common_date)]
    currency_df.reset_index(drop=True, inplace=True)
    
    market_df = market_df[market_df['Date'].isin(common_date)]
    market_df.reset_index(drop=True, inplace=True)
    market_df.loc[:,'Open'] *= currency_df['Price']
    market_df.loc[:,'High'] *= currency_df['Price']
    market_df.loc[:,'Low'] *= currency_df['Price']
    market_df.loc[:,'Price'] *= currency_df['Price']
    market_df.loc[:,'Change'] = market_df['Price'].pct_change()
    
    return market_df

In [12]:
twii_df_new = normalize_to_usd(twii_df,twd_df)
twii_df_new

C:\Users\lzcai\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Date,Open,High,Low,Price,Vol,Change
0,1997-07-02,326.603323,327.681803,322.791520,323.098816,0.0,NaN
1,1997-07-03,325.196469,325.196469,322.896457,323.976195,0.0,0.002716
2,1997-07-04,328.341231,330.971204,327.418139,330.049409,0.0,0.018746
3,1997-07-07,335.669908,335.856995,333.506104,334.150741,0.0,0.012426
4,1997-07-08,334.725996,335.655444,332.453424,333.282068,0.0,-0.002600
...,...,...,...,...,...,...,...
5690,2020-02-20,387.970909,389.355128,385.612956,385.973067,2035400.0,-0.009665
5691,2020-02-21,385.564479,386.683221,383.606246,384.419395,1952700.0,-0.004025
5692,2020-02-24,381.654737,381.654737,378.263786,379.012621,2347100.0,-0.014065
5693,2020-02-25,378.498741,380.483861,375.496521,379.600358,2158400.0,0.001551


In [13]:
bvsp_df_new = normalize_to_usd(bvsp_df,brl_df)
bvsp_df_new

C:\Users\lzcai\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Date,Open,High,Low,Price,Vol,Change
0,1995-01-02,40544.822057,40951.771393,40043.822143,40052.202300,0.0,NaN
1,1995-01-03,40686.390028,40835.359628,38114.802764,38154.838800,0.0,-0.047372
2,1995-01-04,38323.676400,38323.676400,36106.363704,37107.006304,0.0,-0.027463
3,1995-01-05,36854.647870,37527.114915,36637.304900,37493.676485,0.0,0.010420
4,1995-01-06,37512.648975,38153.861342,35433.827700,35567.644549,0.0,-0.051370
...,...,...,...,...,...,...,...
6304,2020-02-18,26474.946400,26474.946400,26066.947200,26398.719200,5548400.0,-0.009351
6305,2020-02-19,26342.376200,26700.459500,26294.723400,26694.273800,6334100.0,0.011196
6306,2020-02-20,26519.496800,26527.235200,26032.660400,26079.773600,6793000.0,-0.023020
6307,2020-02-21,26102.463000,26102.463000,25664.175800,25896.531800,5896200.0,-0.007026


In [14]:
twii_df_new.to_csv("./from github\\Stock-Trading-Environment\\data\\^TWII_new.csv", index=False)
bvsp_df_new.to_csv("./from github\\Stock-Trading-Environment\\data\\^BVSP_new.csv", index=False)

In [76]:
"aaa_new"[-4:]

'_new'

In [52]:
twd_exchange = twd_df[['Date', 'Price']]
common_date = np.intersect1d(twd_exchange['Date'] , twii_df['Date'])

In [53]:
twd_exchange = twd_exchange[twd_exchange['Date'].isin(common_date)]
twd_exchange.reset_index(drop=True, inplace=True)
twii_df = twii_df[twii_df['Date'].isin(common_date)]
twii_df.reset_index(drop=True, inplace=True)
twii_df

,Date,Open,High,Low,Price,Vol,Change
0,1997-07-02,9094.269531,9124.299805,8988.129883,8996.686523,0.0,NaN
1,1997-07-03,9061.599609,9061.599609,8997.509766,9027.596680,0.0,0.003436
2,1997-07-04,9144.959961,9218.209961,9119.250000,9192.536133,0.0,0.018271
3,1997-07-07,9365.190430,9370.410156,9304.820313,9322.805664,0.0,0.014171
4,1997-07-08,9345.549805,9371.500000,9282.099609,9305.235352,0.0,-0.001885
...,...,...,...,...,...,...,...
5690,2020-02-20,11785.780273,11827.830078,11714.150391,11725.089844,2035400.0,-0.002870
5691,2020-02-21,11721.160156,11755.169922,11661.629883,11686.349609,1952700.0,-0.003304
5692,2020-02-24,11615.280273,11615.280273,11512.080078,11534.870117,2347100.0,-0.012962
5693,2020-02-25,11506.740234,11567.089844,11415.469727,11540.230469,2158400.0,0.000465


In [54]:
twii_df['Open'] *= twd_exchange['Price']
twii_df['High'] *= twd_exchange['Price']
twii_df['Low'] *= twd_exchange['Price']
twii_df['Price'] *= twd_exchange['Price']
twii_df['Change'] = twii_df['Price'].pct_change()
twii_df

,Date,Open,High,Low,Price,Vol,Change
0,1997-07-02,326.603323,327.681803,322.791520,323.098816,0.0,NaN
1,1997-07-03,325.196469,325.196469,322.896457,323.976195,0.0,0.002716
2,1997-07-04,328.341231,330.971204,327.418139,330.049409,0.0,0.018746
3,1997-07-07,335.669908,335.856995,333.506104,334.150741,0.0,0.012426
4,1997-07-08,334.725996,335.655444,332.453424,333.282068,0.0,-0.002600
...,...,...,...,...,...,...,...
5690,2020-02-20,387.970909,389.355128,385.612956,385.973067,2035400.0,-0.009665
5691,2020-02-21,385.564479,386.683221,383.606246,384.419395,1952700.0,-0.004025
5692,2020-02-24,381.654737,381.654737,378.263786,379.012621,2347100.0,-0.014065
5693,2020-02-25,378.498741,380.483861,375.496521,379.600358,2158400.0,0.001551
